In [1]:
#move back to the root directory
import os
os.chdir('..')

In [2]:
!pwd

/Users/charlesc/Documents/GitHub/ERP_tools


In [ ]:
import orekit
from orekit.pyhelpers import setup_orekit_curdir
orekit.pyhelpers.download_orekit_data_curdir()
vm = orekit.initVM()
setup_orekit_curdir()

In [53]:
from orekit.pyhelpers import datetime_to_absolutedate
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.hipparchus.linear import RealMatrix
from org.orekit.propagation import StateCovariance
from org.orekit.frames import FramesFactory
from org.orekit.orbits import PositionAngleType, CartesianOrbit, KeplerianOrbit, CircularOrbit, EquinoctialOrbit, OrbitType
from org.orekit.ssa.metrics import ProbabilityOfCollision 
from org.orekit.ssa.collision.shorttermencounter.probability.twod import Patera2005
from org.orekit.utils import PVCoordinates
from org.orekit.utils import Constants
import datetime
import random
from org.hipparchus.linear import MatrixUtils
from org.orekit.utils import IERSConventions, PVCoordinates

In [66]:
from org.hipparchus.linear import MatrixUtils

# Define the covariance matrix values as per your specifications
covariance_values = [
    [127.5401258877026, -216.5469301895778, 23.41184551274494, 0.2482739312599897, -0.1375882369099475, 0.008141795305978281],
    [-216.5469301895778, 9738.762752029239, -5.346171445289681, -10.55496325674788, 0.1222189661319367, -0.05578838789977865],
    [23.41184551274494, -5.346171445289681, 24.60870138594973, 0.005950144116086071, -0.02576925085939057, -0.0003344474012710994],
    [0.2482739312599897, -10.55496325674788, 0.005950144116086071, 0.01147849769311488, -0.0001458466399925159, 0.00005801069817622688],
    [-0.1375882369099475, 0.1222189661319367, -0.02576925085939057, -0.0001458466399925159, 0.0001499094267380701, -0.000008757429553163563],
    [0.008141795305978281, -0.05578838789977865, -0.0003344474012710994, 0.00005801069817622688, -0.000008757429553163563, 0.00009287778622823746]
]

# Convert the Python list to a Java double[][]
jarray = MatrixUtils.createRealMatrix(len(covariance_values), len(covariance_values[0]))
for i in range(len(covariance_values)):
    for j in range(len(covariance_values[i])):
        jarray.setEntry(i, j, covariance_values[i][j])

In [71]:
# Use MatrixUtils to create a RealMatrix object from the 2D array
cov_mat1 = jarray
cov_mat2  = jarray

In [72]:
def symmetrize(matrix):
    nrows, ncols = matrix.getRowDimension(), matrix.getColumnDimension()
    for i in range(nrows):
        for j in range(i+1, ncols):
            value = (matrix.getEntry(i, j) + matrix.getEntry(j, i)) / 2.0
            matrix.setEntry(i, j, value)
            matrix.setEntry(j, i, value)

# Apply symmetrization to your covariance matrices
symmetrize(cov_mat1)
symmetrize(cov_mat2)

In [77]:
def generate_symmetric_covariance_matrix(size):
    # Create an empty matrix of the desired size
    matrix = MatrixUtils.createRealMatrix(size, size)
    
    # Fill the upper triangle and diagonal with random values
    for i in range(size):
        for j in range(i, size):
            value = random.random()
            matrix.setEntry(i, j, value)
            matrix.setEntry(j, i, value)  # Mirror to make it symmetric
    
    return matrix

state1 = [700001.0, 0.0, 0.0, 0.0, 7000.24, 0.0]
state2 = [700000.0, 0.0, 0.0, 0.0, 7000.1, 0.0]
tca = datetime.datetime(2020, 1, 1, 0, 0, 0)

orbit1 = CartesianOrbit(PVCoordinates(Vector3D(float(state1[0]), float(state1[1]), float(state1[2])),
                        Vector3D(float(state1[3]), float(state1[4]), float(state1[5]))),
                        FramesFactory.getEME2000(),
                        datetime_to_absolutedate(tca),
                        Constants.WGS84_EARTH_MU)

orbit2 = CartesianOrbit(PVCoordinates(Vector3D(float(state2[0]), float(state2[1]), float(state2[2])),
                        Vector3D(float(state2[3]), float(state2[4]), float(state2[5]))),
                        FramesFactory.getEME2000(),
                        datetime_to_absolutedate(tca),
                        Constants.WGS84_EARTH_MU)

radius1 = 1.0
radius2 = 1.0

covariance1 = StateCovariance(cov_mat1, datetime_to_absolutedate(tca), FramesFactory.getITRF(IERSConventions.IERS_2010, False), OrbitType.CARTESIAN, PositionAngleType.TRUE)
covariance2 = StateCovariance(cov_mat2, datetime_to_absolutedate(tca), FramesFactory.getITRF(IERSConventions.IERS_2010, False), OrbitType.CARTESIAN, PositionAngleType.TRUE)

# Patera2005.compute(Orbit primaryAtTCA, StateCovariance primaryCovariance, double primaryRadius, Orbit secondaryAtTCA, StateCovariance secondaryCovariance, double secondaryRadius)
patera2005 = Patera2005() 
poc_result = patera2005.compute(orbit1, covariance1, orbit2, covariance2, radius2, 1e-10)
print(f"Probability of collision: {poc_result.getValue()}")

Probability of collision: 0.0005189099581768296


In [ ]:
#state covariance constructor takes in:
#   a 6x6 matrix (RealMatrix)
#   an epoch
#   a local orbital frame

# org.hipparchus.geometry.euclidean.threed.Rotation	rotationFromInertial(AbsoluteDate date, PVCoordinates pv)
# Get the rotation from inertial frame to local orbital frame.